In [1]:
!pip install streamlit, streamlit_chat, openai, langchain, sentence_transformers, openai, unstructured, unstructured[local-inference], pinecone-client

In [2]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader("pro-1.txt")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'pro-1.txt'}, page_content="The Vivobook 15, a multipurpose laptop with a 12th-generation Intel U-Series CPU, DDR4 memory, and PCIe 3.0 SSD storage, delivers remarkable performance for daily work. Its three-sided NanoEdge slim-bezel display offers breathtaking images, and its Dirac-designed audio system produces rich, immersive sound. The laptop has ASUS Antimicrobial Guard Plus for improved cleanliness, a comfortable ASUS ErgoSense keyboard, and rapid charging. In addition, it has a lay-flat hinge for effortless collaboration, a webcam privacy barrier, and several I/O ports for connection. The Vivobook 15 is dependable and fashionable, meeting US MIL-STD-810H military-grade durability requirements with its ASUS AI Noise-Canceling Technology and 3DNR for clearer video conversations.\n\nProduct Description:\nFast-charging Battery Technology\nNever worry about running out of battery during crucial tasks with this laptop's fast-charging technology. This featu

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=300,chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(text_documents)
print(len(docs))


24


In [7]:
docs[13].page_content

"and vibrant visuals whether you're working on presentations, editing photos, or binge-watching shows. The broad viewing angles ensure the display remains clear even when viewed off-centre. Additionally, the TUV Rheinland eye-care certification guarantees low blue-light levels, reducing eye strain"

In [15]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",api_key="")

In [19]:
query="what is its price?"
emb=embeddings.embed_query(query)
len(emb)
## all embeddings will be of size 3072 size only.

3072

In [27]:
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents=docs,embedding=embeddings)

In [32]:
query="what about model name"
retireved_results=vectordb.similarity_search(query)
print(retireved_results[0].page_content)

Specifications
General
Sales Package
Laptop, Power Adaptor, User Guide, Warranty Documents
Model Number
X1504ZA-NJ321WS
Part Number
90NB1021-M00580
Model Name
Vivobook 15
Series
Vivobook 15, with Backlit Keyboard,
Color
Quiet Blue
Type
Thin and Light Laptop
Suitable For
Travel & Business


In [33]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("features?")
rdocs

c:\Users\riddh\Desktop\AI-voice assitant\venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': 'pro-1.txt'}, page_content='ASUS ErgoSense keyboard, and rapid charging. In addition, it has a lay-flat hinge for effortless collaboration, a webcam privacy barrier, and several I/O ports for connection. The Vivobook 15 is dependable and fashionable, meeting US MIL-STD-810H military-grade durability requirements with its ASUS'),
 Document(metadata={'source': 'pro-1.txt'}, page_content='slim-bezel display offers breathtaking images, and its Dirac-designed audio system produces rich, immersive sound. The laptop has ASUS Antimicrobial Guard Plus for improved cleanliness, a comfortable ASUS ErgoSense keyboard, and rapid charging. In addition, it has a lay-flat hinge for effortless'),
 Document(metadata={'source': 'pro-1.txt'}, page_content='Exceptional Processing Power'),
 Document(metadata={'source': 'pro-1.txt'}, page_content='and fashionable, meeting US MIL-STD-810H military-grade durability requirements with its ASUS AI Noise-Canceling Technology and 3DNR 

In [35]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key="")

In [43]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
Try to give as detailed answer as possible, like if user asks about details or features of the product then all the features/details in as much detail as possible.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

# the relevant docs we are pulling for our question is context

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


In [44]:
chain('what are some of its features?')

{'query': 'what are some of its features?',
 'result': 'Some of the features of the ASUS Vivobook 15 include a slim-bezel display for breathtaking images, a Dirac-designed audio system for rich, immersive sound, ASUS Antimicrobial Guard Plus for improved cleanliness, an ASUS ErgoSense keyboard for comfort, rapid charging capability, a lay-flat hinge for effortless collaboration, a webcam privacy barrier, US MIL-STD-810H military-grade durability, lightweight and thin design for portability, versatile companion for various settings, and a range of I/O ports including USB 3.2 Type-A and Type-C ports, HDMI, and an audio combo jack.',
 'source_documents': [Document(metadata={'source': 'pro-1.txt'}, page_content='slim-bezel display offers breathtaking images, and its Dirac-designed audio system produces rich, immersive sound. The laptop has ASUS Antimicrobial Guard Plus for improved cleanliness, a comfortable ASUS ErgoSense keyboard, and rapid charging. In addition, it has a lay-flat hinge 

In [45]:
chain("what about warranty?")

{'query': 'what about warranty?',
 'result': 'The ASUS Vivobook 15 comes with a 1-year warranty with onsite service. The warranty covers manufacturing defects but does not cover physical damage.',
 'source_documents': [Document(metadata={'source': 'pro-1.txt'}, page_content='Warranty\nWarranty Summary\n1-year warranty\nWarranty Service Type\nOnsite\nCovered in Warranty\nManufacturing Defects\nNot Covered in Warranty\nPhysical Damage'),
  Document(metadata={'source': 'pro-1.txt'}, page_content='Windows 11 Home\nDisplay And Audio Features\nTouchscreen\nNo\nScreen Size\n39.62 cm (15.6 Inch)\nScreen Resolution\n1920 x 1080 pixel\nConnectivity Features\nBluetooth\nYES\nAdditional Features\nDisk Drive\nNot Available\nWarranty\nWarranty Summary\n1-year warranty\nWarranty Service Type\nOnsite\nCovered in Warranty'),
  Document(metadata={'source': 'pro-1.txt'}, page_content='the TUV Rheinland eye-care certification guarantees low blue-light levels, reducing eye strain during extended periods of

In [46]:
chain("what is model name, color, screen size, RAM type, storage, battery")

{'query': 'what is model name, color, screen size, RAM type, storage, battery',
 'result': 'Model Name: Vivobook 15\nColor: Quiet Blue\nScreen Size: 39.62 cm (15.6 Inch)\nRAM Type: DDR4\nStorage: SSD\nBattery: Up to 8 hours',
 'source_documents': [Document(metadata={'source': 'pro-1.txt'}, page_content='Color\nQuiet Blue\nType\nThin and Light Laptop\nSuitable For\nTravel & Business\nBattery Backup\nUPTO 8 HRS\nMS Office Provided\nYes\nProcessor And Memory Features\nProcessor Brand\nIntel\nProcessor Name\nCore i3\nProcessor Generation\n12th Gen\nSSD\nYes\nSSD Capacity\n512 GB\nRAM\n8 GB\nRAM Type\nDDR4\nGraphic Processor'),
  Document(metadata={'source': 'pro-1.txt'}, page_content='Processor Generation\n12th Gen\nSSD\nYes\nSSD Capacity\n512 GB\nRAM\n8 GB\nRAM Type\nDDR4\nGraphic Processor\nIntel Integrated Integrated IntelÂ® UHD Graphics\nStorage Type\nSSD\nOperating System\nOperating System\nWindows 11 Home\nDisplay And Audio Features\nTouchscreen\nNo\nScreen Size\n39.62 cm (15.6 Inch)